# Kryptologie Lab 11 - SHA

In [36]:
# helper functions https://stackoverflow.com/a/30375198/6600660

def int_to_bytes(x: int) -> bytes:
    return x.to_bytes((x.bit_length() + 7) // 8, 'big')

def int_from_bytes(xbytes: bytes) -> int:
    return int.from_bytes(xbytes, 'big')

In [37]:
import numpy as np

In [38]:
def initH():
    global h
    h = [0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476, 0xC3D2E1F0]

In [49]:
def extendTo64(mL):
    arr = bytearray(8)
    for i in range(8):
        arr[i] = mL & 0xFF
        mL = mL >> 8
    return arr

def rotateleft(l):
    most_significant = l & 0x80000000
    return (l << 1) | (most_significant >> 31)

def rotateLeftNTimes(l, n):
    for i in range(n):
        l = rotateleft(l)
    return l

def extendTo80(block):
    print(block)
    for i in range(16, 80):
        block.append(rotateleft(block[i-3] ^ block[i-8] ^ block[i-14] ^ block[i-16]))
    return block

def scrumble(extendedBlock):
    a,b,c,d,e = h
    for i in range(80):
        if 0 <= i and i <= 19:
            f = (b & c) | ((0xFFFFFFFF ^ b) & d)
            k = 0x5A827999
        elif 20 <= i and i <= 39:
            f = b ^ c ^ d
            k = 0x6ED9EBA1
        elif 40 <= i and i <= 59:
            f = (b & c) | (b & d) | (c & d)
            k = 0x8F1BBCDC
        elif 60 <= i and i <= 79:
            f = b ^ c ^ d
            k = 0xCA62C1D6
        tmp = (rotateLeftNTimes(a,5) + f + e + k + extendedBlock[i]) & 0xFFFFFFFF
        e = d
        d = c
        c = rotateLeftNTimes(b, 30)
        b = a
        a = tmp

    h[0] = (h[0] + a)  & 0xFFFFFFFF
    h[1] = (h[1] + b) & 0xFFFFFFFF
    h[2] = (h[2] + c) & 0xFFFFFFFF
    h[3] = (h[3] + d) & 0xFFFFFFFF
    h[4] = (h[4] + e) & 0xFFFFFFFF
    


def hash(m):
    mL = len(m) * 8
    m += bytes([0x80])
    while (len(m) % 64) != (56 % 64):
        m += bytes([0])
    m += extendTo64(mL)

    blocks = []
    for i in range(0, len(m), 64):
        inner_block = []
        for j in range(0, 64, 4):
            inner_block.append(int_from_bytes(m[i+j:i+j+4]))
        blocks.append(inner_block)

    initH()
    for i in range(len(blocks)):
        blocks[i] = scrumble(extendTo80(blocks[i]))

    return hex(h[0] << 128 | h[1] << 96 | h[2] << 64 | h[3] << 32 | h[4])



In [50]:
hash(b"")

[2147483648, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


'0xda39a3ee5e6b4b0d3255bfef95601890afd80709'